In [1]:
import pandas as pd
import plotly.graph_objects as go
import requests
from tqdm.auto import tqdm

In [2]:
def hours_in_month(month: str):
    html = requests.get(f"https://www.currentresults.com/Weather/Europe/Cities/sunshine-average-{month}.php").text
    tables = pd.read_html(html)
    concatenated = pd.concat(tables, ignore_index=True)
    split = concatenated["City"].str.split(", ", expand=True).rename({0: "City", 1: "Country"}, axis="columns")
    joined = pd.concat([concatenated.drop("City", axis="columns"), split], axis="columns")
    return joined.set_index(["Country", "City"])

hours_in_month("january").loc["Italy"]

,Hours
City,
Milan,59
Naples,115
Turin,112


In [3]:
all_months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
sunshine_data = pd.concat([hours_in_month(month).rename({"Hours": month}, axis="columns") for month in tqdm(all_months)], axis="columns")
sunshine_data.loc["Italy"]

  0%|          | 0/12 [00:00<?, ?it/s]

,january,february,march,april,may,june,july,august,september,october,november,december
City,,,,,,,,,,,,
Milan,59,95,152,177,211,243,285,251,186,130,66,59
Naples,115,123,158,189,245,279,313,295,234,189,126,105
Turin,112,118,158,180,195,219,260,223,168,143,105,109


In [4]:
sunshine_data.index.get_level_values(1)

Index(['Amsterdam', 'Andorra la Vella', 'Athens', 'Barcelona', 'Belgrade',
       'Berlin', 'Birmingham', 'Brussels', 'Bucharest', 'Budapest', 'Chisinau',
       'Cologne', 'Copenhagen', 'Dublin', 'Glasgow', 'Hamburg', 'Helsinki',
       'İstanbul', 'Kazan', 'Lille', 'Lisbon', 'Ljubljana', 'London',
       'Luxembourg', 'Lyon', 'Madrid', 'Manchester', 'Marseille', 'Milan',
       'Minsk', 'Monaco', 'Moscow', 'Munich', 'Naples', 'Nice',
       'Nizhny & Novgorod', 'Odessa', 'Oslo', 'Paris', 'Podgorica', 'Prague',
       'Pristina', 'Reykjavík', 'Riga', 'Rostov-on-Don', 'Rotterdam',
       'Saint Petersburg', 'Sarajevo', 'Skopje', 'Sofia', 'Stockholm',
       'Tallinn', 'Turin', 'Vaduz', 'Valletta', 'Vienna', 'Vilnius', 'Zagreb',
       'Zurich'],
      dtype='object', name='City')

In [5]:
def darkest_plot():
    darkest = sunshine_data.min(axis="columns").sort_values() / 30
    return go.Figure(
        layout=dict(
            title="Sunshine in the darkest month",
            xaxis_title="Hours of sun per day",
            height=1024,
            template="plotly_dark",
        ),
        data=[
            go.Bar(
                orientation="h",
                x=darkest,
                y=darkest.index.get_level_values(1),
                marker_color=darkest,
            )
        ]
    )

darkest_plot()